In [2]:
import torch
import numpy as np
import torch.nn as nn


In [71]:
def get_jacobian(net, x, noutputs):
    x = x.squeeze()
    n = x.size()[0]
    x = x.repeat(noutputs, 1)
    x.requires_grad_(True)
    print(x.shape)
    y = net(x)
    mask = torch.Tensor((1, 1, 1, 1, 1, 1))
    y[0].backward(torch.diag(mask), create_graph=True)
    
    return x.grad

In [61]:
class Net(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(Net, self).__init__()

        self.layer1 = nn.Linear(input_size, hidden_size, bias=False)
        nn.init.xavier_uniform(self.layer1.weight.data)
        self.input_size = input_size

    def forward(self, input, other_input):
        x = self.layer1(input)
        
        x1 = self.layer1(x[:, :self.input_size])
        x2 = self.layer1(x[:, self.input_size:])
        
        return x1, x2

In [62]:
net = Net(3, 6, 3)

/Users/michelle/miniconda3/envs/guapo/lib/python3.6/site-packages/ipykernel_launcher.py:6: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
  


In [66]:
x = np.array((1, 1, 1))
x = np.expand_dims(x, 0)
x = torch.from_numpy(x).float()

print(net.layer1.weight.data@ net.layer1.weight.data.transpose(-1,-2))

tensor([[ 0.6572,  0.3552,  0.0864, -0.3099, -0.1776, -0.1809],
        [ 0.3552,  1.1654,  0.2338, -0.1073,  0.0512, -0.4135],
        [ 0.0864,  0.2338,  0.1620, -0.0108,  0.1773,  0.1525],
        [-0.3099, -0.1073, -0.0108,  0.1528,  0.1205,  0.1038],
        [-0.1776,  0.0512,  0.1773,  0.1205,  0.3296,  0.3576],
        [-0.1809, -0.4135,  0.1525,  0.1038,  0.3576,  0.6422]])


In [67]:
net(x)

(tensor([[ 0.4059,  1.1081,  0.2779, -0.1279,  0.1054, -0.2891]],
        grad_fn=<MmBackward>),
 tensor([[-0.0388,  0.8516,  0.4464,  0.1178,  0.5693,  0.3144]],
        grad_fn=<MmBackward>))

In [72]:
jacobian = get_jacobian(net, x, 6)
print(jacobian)

torch.Size([6, 3])
tensor([[ 0.0398, -0.1171,  0.4832],
        [ 0.0431,  0.3879,  0.6771],
        [-0.2643,  0.3100,  0.2321],
        [-0.0613,  0.1214, -0.1879],
        [-0.4191,  0.4588,  0.0656],
        [-0.5837,  0.3793, -0.0847]], grad_fn=<CloneBackward>)


In [59]:
jacobian.requires_grad

True

In [73]:
t1 = torch.zeros((5, 3, 2))
t2 = torch.zeros((5, 2, 4))

In [75]:
torch.bmm(t1, t2).shape

torch.Size([5, 3, 4])